# Import Packages & Define functions

In [1]:
# read csv
import pandas as pd
import os
# PdfMiner
import glob
import numpy as np
from io import StringIO
from io import BytesIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
# Token Vectorization
from langdetect import detect 
import fasttext.util
import fasttext
########################################
from random import random
from numpy import array
from numpy import cumsum
########################################
import regex as re  
import string
import re
########################################
from datetime import datetime
import collections
########################################
# Keras imports for ML Model
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, Bidirectional
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D, Embedding
from keras.utils import np_utils
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

from dataclasses import dataclass
from bs4 import BeautifulSoup

Using TensorFlow backend.


In [2]:
def read_tei(tei_file):
    with open(tei_file, 'r',encoding="utf-8") as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')

In [5]:
def elem_to_text(elem, default=''):
    if elem:
        return elem.getText()
    else:
        return default

In [4]:


@dataclass
class Person:
    firstname: str
    middlename: str
    surname: str

turing_author = Person(firstname='Alan', middlename='M', surname='Turing')

f"{turing_author.firstname} {turing_author.surname} authored many influential publications in computer science."

'Alan Turing authored many influential publications in computer science.'

In [6]:
class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)
        self._text = None
        self._title = ''
        self._abstract = ''

    @property
    def doi(self):
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return ''
        else:
            return idno_elem.getText()

    @property
    def title(self):
        if not self._title:
            self._title = self.soup.title.getText()
        return self._title

    @property
    def abstract(self):
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True)
            self._abstract = abstract
        return self._abstract

    @property
    def authors(self):
        authors_in_header = self.soup.analytic.find_all('author')

        result = []
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename", type="first"))
            middlename = elem_to_text(persname.find("forename", type="middle"))
            surname = elem_to_text(persname.surname)
            person = Person(firstname, middlename, surname)
            result.append(person)
        return result
    
    @property
    def text(self):
        if not self._text:
            divs_text = []
            for div in self.soup.body.find_all("div"):
                # div is neither an appendix nor references, just plain text.
                if not div.get("type"):
                    div_text = div.get_text(separator=' ', strip=True)
                    divs_text.append(div_text)

            plain_text = " ".join(divs_text)
            self._text = plain_text
        return self._text

In [7]:
# Function that get the fullpath of files in a directory
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [8]:
### Funktion zum entfernen von Zeilenumbrüchen
def removePassage(my_str):
    my_str1 = re.sub("\\\\ud", " ", my_str)
    my_str2 = re.sub("\\\\n", " ", my_str1)
    return(my_str2)

### Funktion zum parsen von PDF[nur erste Seite] zu String Format
def extract_page_one(path):
    output_string = StringIO()
    
    with open(path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        interpreter.process_page(list(PDFPage.create_pages(doc))[0])
        return(output_string)
    


### Funktion zum Vergrößern der Input & Output Vektoren um NEWLINES
def add_newlines(Tokens,Real_Tokens,y_final):
    y_final_REAL = []
    k = 0
    m = 0
    for i in range(len(Tokens)):
        if k == 0:
            j=i
        else:
            if m == 0:
                j = k+1
            else:
                j = m+1
        if Tokens[i] == Real_Tokens[j] : # Wenn Tokens gleich sind, dann übernehme y_final_REAL
            y_final_REAL.append(y_final[i])
            m = j
        else:
            for k in range(j,len(Real_Tokens)): # Sonst gehe die Real_Tokens durch bis match vorhanden

                if Real_Tokens[k] == 'NEWLINE':
                    y_final_REAL.append('Sonstiges')
                
#                 elif Real_Tokens[k] not in Tokens:
#                     y_final_REAL.append('Sonstiges')

                else:
                    y_final_REAL.append(y_final[i])
                    m=k
                    break

    RealTokens_final = Real_Tokens[:len(y_final_REAL)]
    
    index_title = [i for i, e in enumerate(y_final_REAL) if e == 'I-title']
    if index_title==[]:
        return(RealTokens_final,y_final_REAL)
    else:
        end_title = max(index_title)

        ### lable NEWLINES im Titel als "I-title"        
        for i in range(len(RealTokens_final)):
            if RealTokens_final[i]=='NEWLINE':
                if (y_final_REAL[i+1] =='I-title' or end_title>i) and y_final_REAL[i-1] in ('B-title','I-title'):
                    y_final_REAL[i] = 'I-title'
        ## Kann passieren, dass im Titel mehrere NEWLINES aufeinander folgen. Daher end_title>i um festzustellen, ob iwann später noch ein Titel Label kommt        

        return(RealTokens_final,y_final_REAL)

#Create Word Vector representation
def detect_and_vectorize(tokens_sequence): #### input is the tokens list of ONE PAPER e.g. ergebnis_tokens[2]
    
    tokens_vectorized = []
    lang = detect(' '.join(tokens_sequence))
    
    if (lang == 'ru'):
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_ru.get_word_vector(tokens_sequence[i])))
                
    elif (lang == 'bg'):
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_bg.get_word_vector(tokens_sequence[i])))
                
    else:  ## assume language == uk
        for i in range(len(tokens_sequence)):
            tokens_vectorized.append(np.float16(ft_uk.get_word_vector(tokens_sequence[i])))
    
    while len(tokens_vectorized)<1000:
        tokens_vectorized.append(np.zeros(60))
      
    if len(tokens_vectorized)>1000:
        del tokens_vectorized[1000:] 

    return np.array(tokens_vectorized)

### Additional Features
punctuations = '''!()[]{};:'"\<>/?@#$%^&*«»_–~.,-'''

def compute_additional_features(tokens_sequence): #### input is the tokens list of ONE PAPER e.g. ergebnis_tokens[2]
    
    tokens = tokens_sequence
    feature_upper = []
    feature_capitalized = []
    feature_autor_format = []
    feature_punctation = []
    feature_newline = []
    feature_array = []
    
    while len(tokens)<1000:
        tokens.append(str(0))
    if len(tokens)>1000:
        del tokens[1000:] 
    #print(tokens)    
    for i in range(len(tokens)):
        if tokens[i] !='NEWLINE':
            if str(tokens[i]).isupper():
                feature_upper.append(1)

            else:
                feature_upper.append(0)
        else: 
            feature_upper.append(0)

        if tokens[i] !='NEWLINE':
            if str(tokens[i][0]).isupper():
                feature_capitalized.append(1)

            else:
                feature_capitalized.append(0)
        else: 
            feature_capitalized.append(0)

        if tokens[i] !='NEWLINE':
            if re.match('.\.',str(tokens[i])) != None and str(tokens[i]).isupper():
                feature_autor_format.append(1)

            else:
                feature_autor_format.append(0)
        else: 
            feature_autor_format.append(0)

        if tokens[i] !='NEWLINE':
            if any((c in punctuations) for c in str(tokens[i])):
                feature_punctation.append(1)
            else:
                feature_punctation.append(0)
        else: 
            feature_punctation.append(0)
                
        if tokens[i] =='NEWLINE':
            feature_newline.append(1)
        else: 
            feature_newline.append(0)
    df = pd.DataFrame(list(zip(feature_upper, feature_capitalized,feature_autor_format ,feature_punctation,feature_newline)))  
    feature_array = df.to_numpy(copy=True)
    
    return np.array(feature_array)

In [9]:
### Grobid: Entferne [[...]]
### Funktion zum entfernen von 'Arrays' aus Autoren
def removeAutor_grobid(my_str):
    my_str1 = re.sub("\[\['", "", my_str) 
    my_str2 = re.sub("'\]\]", "", my_str1)
    my_str3 = re.sub("'", "", my_str2) 
    return(my_str3)

In [10]:
### Funktion zum entfernen von 'Arrays' aus Autoren
def removeAutor(my_str):
    my_str1 = re.sub("\['", "", my_str)
    my_str2 = re.sub("'\]", "", my_str1)
    my_str3 = re.sub("'", "", my_str2)
    return(my_str3)


# Read TEI Files

In [10]:
### DO NOT RUN! Predictions of GROBID are already saved in 'grobid_16467.csv'
### Transform the tei.xml files into a dataframe


all_files = listdir_fullpath("D:\_final_selection_16478\GROBID_ALL")
# all_files = listdir_fullpath("D:\_final_selection_16478\out2")

Frame = pd.DataFrame({"core_id": [] , "title": [] , "authors": [] })
i = 0
for tei_doc in all_files:
    tei = TEIFile(tei_doc)
    core_id = re.sub(".tei.xml","",re.sub("D:\\\\_final_selection_16478\\\\GROBID_ALL\\\\Core_ID_","",tei_doc))
    authors = []
    for i in range(len(tei.authors)):
        if len(tei.authors[i].firstname)==1:
            forename = tei.authors[i].firstname + ". " + tei.authors[i].middlename + "."
            surname = tei.authors[i].surname
            name = [forename , surname]
        elif len(tei.authors[i].middlename)==1:
            forename = tei.authors[i].firstname +" " +tei.authors[i].middlename + "."
            surname = tei.authors[i].surname
            name = [forename , surname]
        else:
            forename = tei.authors[i].firstname + " " +tei.authors[i].middlename
            surname = tei.authors[i].surname
            name = [forename , surname]
        authors.append(name)
    
    
    Frame = Frame.append(pd.DataFrame(data = {"core_id": core_id , "title":tei.title , "authors":str(authors)},index = [i]), ignore_index=True)
    i =+ 1

# Frame.to_excel("grobid_0_8000.xlsx")
# Frame.to_excel("grobid_8000_16478.xlsx")
Frame.to_excel("grobid_16467.xlsx")
Frame.to_csv('grobid_16467.csv')
Frame

,core_id,title,authors
0,11083759,Середовище проведення освітніх вебінарів «WIP ...,"[['Юрій ', 'Богачков']]"
1,11083794,Комплексна підготовка дистанційних матеріалів ...,[]
2,11083797,,[]
3,11083801,,[]
4,11083807,Інструментальні засоби профорієнтації,[]
...,...,...,...
16462,95313001,,[]
16463,95313006,,[]
16464,95313010,,[]
16465,95313012,ТЕХНОЛОГІЯ ПОКВАРТИРНОГО ОБЛІКУ ТЕПЛОВОЇ ЕНЕРГІЇ,"[['Горбачьова А О', '']]"


# Evaluate GROBID prediction

In [11]:
df_meta = pd.read_csv('grobid_16467.csv', sep = ',')#  , encoding= 'utf-16')
df_meta.drop('Unnamed: 0' , axis = 1 , inplace=True)
df_meta

,core_id,title,authors
0,11083759,Середовище проведення освітніх вебінарів «WIP ...,"[['Юрій ', 'Богачков']]"
1,11083794,Комплексна підготовка дистанційних матеріалів ...,[]
2,11083797,NaN,[]
3,11083801,NaN,[]
4,11083807,Інструментальні засоби профорієнтації,[]
...,...,...,...
16462,95313001,NaN,[]
16463,95313006,NaN,[]
16464,95313010,NaN,[]
16465,95313012,ТЕХНОЛОГІЯ ПОКВАРТИРНОГО ОБЛІКУ ТЕПЛОВОЇ ЕНЕРГІЇ,"[['Горбачьова А О', '']]"


In [16]:
path_15553 = "D:/_final_selection_16478/PDFs_15553/"


pdf = os.listdir(path_15553)


files_core_id = []
files_paths = []
for elem in pdf:
    core = int(re.sub(".pdf","",re.sub("Core_ID_","",elem)))
    if core in list(df_meta.core_id):
        files_core_id.append(core)
#         files_paths.append("D:/_final_selection_16478/all_pdf/" + elem)
        files_paths.append(path_15553 + elem)
print(len(files_paths))
print(files_paths[0])



15552
D:/_final_selection_16478/PDFs_15553/Core_ID_11083759.pdf


In [13]:
df_meta = df_meta[df_meta.core_id.isin(files_core_id)].reset_index()
df_meta.drop('index' , axis = 1 , inplace=True)
print(df_meta.shape)
print("{} Titel und {} Autoren sind NA".format(sum(df_meta.title.isna()),sum(df_meta.authors == "[]")))

(15552, 3)
12787 Titel und 14910 Autoren sind NA


In [15]:
### extract text 

all_pdf_text = [] 
start_time = datetime.now()
for i in range(len(files_paths)):
    try:
        all_pdf_text.append(extract_page_one(files_paths[i]).getvalue())
        if i % 500 == 0:
            print(str((i/len(files_paths))*100)+'%')
    except:
        all_pdf_text.append("Einlesen nicht moeglich")
        print("FEHLER")
    
end_time = datetime.now()

0.0%
3.2150205761316872%
6.4300411522633745%
9.645061728395062%
12.860082304526749%
16.075102880658438%
19.290123456790123%
22.505144032921812%
25.720164609053498%
28.935185185185187%
32.150205761316876%
35.365226337448554%
38.58024691358025%
41.79526748971193%
45.010288065843625%
48.2253086419753%
51.440329218106996%
54.65534979423868%
57.870370370370374%
61.08539094650206%
64.30041152263375%
67.51543209876543%
70.73045267489711%
73.9454732510288%
77.1604938271605%
80.37551440329219%
83.59053497942386%
86.80555555555556%
90.02057613168725%
93.23559670781893%
96.4506172839506%
99.6656378600823%


In [18]:
### Import REAL Meta Data
path_meta = 'D:/_final_selection_16478/'

df_meta_real = pd.read_csv(path_meta + 'final_items_15553.csv', sep = ',')#  , encoding= 'utf-16')
# df_meta.drop('Unnamed: 0' , axis = 1 , inplace=True)
# df_meta_real

### get all titles from meta data with core_ids of fulltext
titles_real = []
for i in range(len(files_core_id)):
    index = df_meta_real.index[df_meta_real['coreId'] == int(files_core_id[i])].tolist()
    if index == []:
        titles_real.append('Keine Meta Daten gefunden')
    else: 
        index = index[0]
        title_pdf  = df_meta_real.loc[index,'title']
        titles_real.append(title_pdf)
len(titles_real)

### Get autor for the PDF´s
######## PROBLEM: verschiedene Schreibweisen Meta <-> PDF
autors_real = []
for i in range(len(files_core_id)):
    index = df_meta_real.index[df_meta_real['coreId'] == int(files_core_id[i])].tolist()
    index = index[0]
    autor_pdf  = df_meta_real.loc[index,'authors']

    autor_pdf = removeAutor(autor_pdf).split(",")
    for j in range(len(autor_pdf)):
        autor_pdf[j] = ' '.join(autor_pdf[j].split()) ## Entferne überflüssige Whitespaces (auch am Anfang)
        
    autors_real.append(autor_pdf)
len(autors_real)

15552

# Get Label vectors for the GROBID Prediction

In [19]:
### get all titles from meta data with core_ids of fulltext
titles = []
for i in range(len(df_meta.core_id)):
    index = df_meta.index[df_meta['core_id'] == int(df_meta.core_id[i])].tolist()
    if df_meta.title.isna()[i]:
        titles.append('Keine Meta Daten gefunden')
    else: 
        index = index[0]
        title_pdf  = df_meta.loc[index,'title']
        titles.append(title_pdf)

len(titles)

15552

In [20]:
### Get autor for the PDF´s
######## PROBLEM: verschiedene Schreibweisen Meta <-> PDF
autors = []
for i in range(len(df_meta.core_id)):
    index = df_meta.index[df_meta['core_id'] == int(df_meta.core_id[i])].tolist()
    index = index[0]
    autor_pdf  = df_meta.loc[index,'authors']

    autor_pdf = removeAutor_grobid(autor_pdf).split(",")
    for j in range(len(autor_pdf)):
        autor_pdf[j] = ' '.join(autor_pdf[j].split()) ## Entferne überflüssige Whitespaces (auch am Anfang)
        
    autors.append(autor_pdf)
len(autors)

15552

In [21]:
### Funktion, die die Daten labelt 

kein_autor = []
kein_titel = []
error_papers = []
ergebnis_tokens = []
ergebnis_label = []
### Real Meta
ergebnis_label_real = []
###

anzahl_papers = len(df_meta.core_id)

for paper in range(anzahl_papers):

    title = ' '.join(removePassage(titles[paper]).split()).lower() # Remove excces Whitespace & to lowercase
    title = re.sub("\(","\(",title) # () as non-regex string
    title = re.sub("\)","\)",title)
    title = re.sub("\*","\*",title) # * as non-regex string

    title_index = re.search(title, ' '.join(all_pdf_text[paper].split()).lower()) # search for the title
    
    #### Real Meta
    title_real = ' '.join(removePassage(titles_real[paper]).split()).lower() # Remove excces Whitespace & to lowercase
    title_real = re.sub("\(","\(",title_real) # () as non-regex string
    title_real = re.sub("\)","\)",title_real)
    title_real = re.sub("\*","\*",title_real) # * as non-regex string

    title_index_real = re.search(title_real, ' '.join(all_pdf_text[paper].split()).lower()) # search for the title
    ######
    
    print('CoreID:  ' + str(files_core_id[paper]))

    if title_index==None:
        Text_pdf_0 = ' '.join(all_pdf_text[paper].split())
        
        kein_titel.append(df_meta.core_id[paper])
        y_final= np.repeat('Sonstiges',len(Text_pdf_0.split()))
    else:
        
        Text_pdf_0 = ' '.join(all_pdf_text[paper].split())

        ##### TITLE ################################################
        if title_index.start()==0:
            teil_B = ""
        else:
            teil_B = Text_pdf_0[0:title_index.start()-1]
        teil_T = Text_pdf_0[title_index.start():title_index.end()]
        teil_E = Text_pdf_0[title_index.end()+1:len(Text_pdf_0)]

        y_teil1 = np.repeat('Sonstiges',len(teil_B.split()))
        y_teil2 = np.append(['B-title'],np.repeat('I-title',len(teil_T.split())-1))
        y_teil3 = np.repeat('Sonstiges',len(teil_E.split()))

        y_final = np.concatenate((y_teil1, y_teil2 , y_teil3), axis=None)
        
    ### Real Meta    
    if title_index_real!=None:
        Text_pdf_0 = ' '.join(all_pdf_text[paper].split())

        ##### TITLE ################################################
        if title_index_real.start()==0:
            teil_B_real = ""
        else:
            teil_B_real = Text_pdf_0[0:title_index_real.start()-1]
        teil_T_real = Text_pdf_0[title_index_real.start():title_index_real.end()]
        teil_E_real = Text_pdf_0[title_index_real.end()+1:len(Text_pdf_0)]

        y_teil1_r = np.repeat('Sonstiges',len(teil_B_real.split()))
        y_teil2_r = np.append(['B-title'],np.repeat('I-title',len(teil_T_real.split())-1))
        y_teil3_r = np.repeat('Sonstiges',len(teil_E_real.split()))

        y_final_real = np.concatenate((y_teil1_r, y_teil2_r , y_teil3_r), axis=None)
    ###
    
    ##### Get Text
    all_pdf_text1 = re.sub("\\n"," NEWLINE ",all_pdf_text[paper])
    Text_pdf_0_NL = ' '.join(all_pdf_text1.split())

    Tokens = Text_pdf_0.split()
    Labels = y_final
    Real_Tokens = Text_pdf_0_NL.split()

    Tokens = all_pdf_text[paper].split()

    Tokens_final_lower = []
    for i in range(len(Tokens)):
        Tokens_final_lower.append(Tokens[i].lower())
    try:
        if autors[paper]!= ['[]']:

            autors_surname = []
            for i in range(len(autors[paper])):
                if i % 2 == 0:
                    autors_surname.append(autors[paper][i])

            autors_surname_lower = []
            for i in range(len(autors_surname)):
                autors_surname_lower.append(autors_surname[i].lower())

            if re.match('.\.',autors[paper][1]) == None:
                autors_forename = []
                for i in range(len(autors[paper])):
                    if i % 2 == 1:
                        autors_forename.append(autors[paper][i].split())

                autors_forename = list(np.concatenate((autors_forename), axis=None))
                autors_forename_lower = []
                for i in range(len(autors_forename)):
                    autors_forename_lower.append(autors_forename[i].lower())

                autors_surname_lower = list(np.concatenate((autors_forename_lower,autors_surname_lower), axis=None))


            vec_autor = []
            for token in Tokens_final_lower:
                line = any(word in token for word in autors_surname_lower)
                vec_autor.append(line)

            index_autor = [i for i, e in enumerate(vec_autor) if e == True]

            if title_index!=None:
                if len(index_autor)>(len(autors_surname_lower)):
                    diff = len(index_autor) - len(autors_surname_lower)
                    dist = []
                    for j in range(len(index_autor)):
                        dist.append(abs(index_autor[j]-np.where(y_final=="B-title")[0][0]))

                    dict1 = dict(zip(dist , index_autor))

                    dist.sort(reverse = True)

                    for k in range(len(dist[0:diff])):
                        vec_autor[dict1[dist[0:diff][k]]] = False

            for i in range(len(y_final)):
                if vec_autor[i] == True:
                    y_final[i] = 'autor'

            if True not in vec_autor:
                kein_autor.append(files_core_id[paper])

            if re.match('.\.',autors[paper][1]) != None:

                index_autor_true = [i for i, e in enumerate(vec_autor) if e == True]

                for w in range(len(index_autor_true)):
                    index = index_autor_true[w]
                    for t in range(index - 4,index + 4):
                        if re.match('.\.',Tokens_final_lower[t]) != None and Tokens[t].isupper():
                            y_final[t] = 'autor'
        ### Real Meta
        autors_surname_real = []
        for i in range(len(autors_real[paper])):
            if i % 2 == 0:
                autors_surname_real.append(autors_real[paper][i])

        autors_surname_lower_real = []
        for i in range(len(autors_surname_real)):
            autors_surname_lower_real.append(autors_surname_real[i].lower())

        if re.match('.\.',autors_real[paper][1]) == None:
            autors_forename_real = []
            for i in range(len(autors_real[paper])):
                if i % 2 == 1:
                    autors_forename_real.append(autors_real[paper][i].split())

            autors_forename_real = list(np.concatenate((autors_forename_real), axis=None))
            autors_forename_lower_real = []
            for i in range(len(autors_forename_real)):
                autors_forename_lower_real.append(autors_forename_real[i].lower())

            autors_surname_lower_real = list(np.concatenate((autors_forename_lower_real,autors_surname_lower_real), axis=None))


        vec_autor_real = []
        for token in Tokens_final_lower:
            line_real = any(word in token for word in autors_surname_lower_real)
            vec_autor_real.append(line_real)

        index_autor_real = [i for i, e in enumerate(vec_autor_real) if e == True]

        if title_index_real!=None:
            if len(index_autor_real)>(len(autors_surname_lower_real)):
                diff = len(index_autor_real) - len(autors_surname_lower_real)
                dist = []
                for j in range(len(index_autor_real)):
                    dist.append(abs(index_autor_real[j]-np.where(y_final_real=="B-title")[0][0]))

                dict1 = dict(zip(dist , index_autor_real))

                dist.sort(reverse = True)

                for k in range(len(dist[0:diff])):
                    vec_autor_real[dict1[dist[0:diff][k]]] = False

        for i in range(len(y_final_real)):
            if vec_autor_real[i] == True:
                y_final_real[i] = 'autor'

#         if True not in vec_autor_real:
#             kein_autor.append(files_core_id[paper])

        if re.match('.\.',autors_real[paper][1]) != None:

            index_autor_true_real = [i for i, e in enumerate(vec_autor_real) if e == True]

            for w in range(len(index_autor_true_real)):
                index = index_autor_true_real[w]
                for t in range(index - 4,index + 4):
                    if re.match('.\.',Tokens_final_lower[t]) != None and Tokens[t].isupper():
                        y_final_real[t] = 'autor'
        ###

        RealTokens_final = add_newlines(Tokens,Real_Tokens,y_final)[0]
        y_final_REAL = add_newlines(Tokens,Real_Tokens,y_final)[1]
        ### Real Meta
        y_final_REAL2 = add_newlines(Tokens,Real_Tokens,y_final_real)[1]
        ergebnis_label_real.append(y_final_REAL2)
        ###
        ergebnis_label.append(y_final_REAL)
        ergebnis_tokens.append(RealTokens_final)
    except:
        error_papers.append(files_core_id[paper])

CoreID:  11083759
CoreID:  11083794
CoreID:  11083801
CoreID:  11083807
CoreID:  11083814
CoreID:  11083816
CoreID:  11083818
CoreID:  11083820
CoreID:  11083841
CoreID:  11083843
CoreID:  11084183
CoreID:  11084185
CoreID:  11084196
CoreID:  11084226
CoreID:  11084228
CoreID:  11084232
CoreID:  11084286
CoreID:  11084289
CoreID:  11084295
CoreID:  11084334
CoreID:  11084432
CoreID:  11084498
CoreID:  11084525
CoreID:  11084527
CoreID:  11084531
CoreID:  11084535
CoreID:  11084639
CoreID:  11084670
CoreID:  11084672
CoreID:  11084674
CoreID:  11084676
CoreID:  11084682
CoreID:  11084739
CoreID:  11084741
CoreID:  11311954
CoreID:  11311956
CoreID:  11311957
CoreID:  11312082
CoreID:  11312083
CoreID:  11312155
CoreID:  11312159
CoreID:  11312161
CoreID:  11312162
CoreID:  11312164
CoreID:  11312165
CoreID:  11312167
CoreID:  11312168
CoreID:  11312169
CoreID:  11312171
CoreID:  11312173
CoreID:  11312174
CoreID:  11312176
CoreID:  11312178
CoreID:  11312180
CoreID:  11312182
CoreID:  1

CoreID:  11313487
CoreID:  11313488
CoreID:  11313489
CoreID:  11313490
CoreID:  11313491
CoreID:  11313492
CoreID:  11313494
CoreID:  11313495
CoreID:  11313496
CoreID:  11313498
CoreID:  11313499
CoreID:  11313501
CoreID:  11313503
CoreID:  11313504
CoreID:  11313517
CoreID:  11313526
CoreID:  11313533
CoreID:  11313534
CoreID:  11313537
CoreID:  11313540
CoreID:  11313541
CoreID:  11313550
CoreID:  11313559
CoreID:  11313562
CoreID:  11313563
CoreID:  11313582
CoreID:  11313589
CoreID:  11313592
CoreID:  11313595
CoreID:  11313599
CoreID:  11313601
CoreID:  11313602
CoreID:  11313605
CoreID:  11313627
CoreID:  11313631
CoreID:  11313640
CoreID:  11313641
CoreID:  11313658
CoreID:  11313668
CoreID:  11313669
CoreID:  11313671
CoreID:  11313672
CoreID:  11313673
CoreID:  11313674
CoreID:  11313675
CoreID:  11313676
CoreID:  11313677
CoreID:  11313678
CoreID:  11313679
CoreID:  11313680
CoreID:  11313681
CoreID:  11313682
CoreID:  11313683
CoreID:  11313684
CoreID:  11313685
CoreID:  1

CoreID:  11314362
CoreID:  11314363
CoreID:  11314364
CoreID:  11314365
CoreID:  11314366
CoreID:  11314367
CoreID:  11314368
CoreID:  11314369
CoreID:  11314370
CoreID:  11314371
CoreID:  11314372
CoreID:  11314374
CoreID:  11314375
CoreID:  11314377
CoreID:  11314378
CoreID:  11314379
CoreID:  11314380
CoreID:  11314381
CoreID:  11314382
CoreID:  11314383
CoreID:  11314389
CoreID:  11314390
CoreID:  11314391
CoreID:  11314392
CoreID:  11314393
CoreID:  11314394
CoreID:  11314395
CoreID:  11314396
CoreID:  11314397
CoreID:  11314398
CoreID:  11314399
CoreID:  11314400
CoreID:  11314401
CoreID:  11314402
CoreID:  11314403
CoreID:  11314404
CoreID:  11314405
CoreID:  11314406
CoreID:  11314407
CoreID:  11314408
CoreID:  11314409
CoreID:  11314410
CoreID:  11314411
CoreID:  11314412
CoreID:  11314413
CoreID:  11314414
CoreID:  11314415
CoreID:  11314416
CoreID:  11314417
CoreID:  11314418
CoreID:  11314419
CoreID:  11314420
CoreID:  11314421
CoreID:  11314422
CoreID:  11314423
CoreID:  1

CoreID:  11315520
CoreID:  11315521
CoreID:  11315522
CoreID:  11315523
CoreID:  11315524
CoreID:  11315525
CoreID:  11315526
CoreID:  11315527
CoreID:  11315528
CoreID:  11315529
CoreID:  11315530
CoreID:  11315532
CoreID:  11315533
CoreID:  11315535
CoreID:  11315536
CoreID:  11315537
CoreID:  11315538
CoreID:  11315539
CoreID:  11315540
CoreID:  11315541
CoreID:  11315542
CoreID:  11315544
CoreID:  11315545
CoreID:  11315546
CoreID:  11315547
CoreID:  11315548
CoreID:  11315549
CoreID:  11315550
CoreID:  11315551
CoreID:  11315552
CoreID:  11315554
CoreID:  11315555
CoreID:  11315556
CoreID:  11315557
CoreID:  11315558
CoreID:  11315559
CoreID:  11315560
CoreID:  11315561
CoreID:  11315562
CoreID:  11315563
CoreID:  11315564
CoreID:  11315565
CoreID:  11315566
CoreID:  11315567
CoreID:  11315568
CoreID:  11315569
CoreID:  11315570
CoreID:  11315571
CoreID:  11315572
CoreID:  11315576
CoreID:  11315577
CoreID:  11315578
CoreID:  11315579
CoreID:  11315580
CoreID:  11315581
CoreID:  1

CoreID:  11320268
CoreID:  11320291
CoreID:  11320302
CoreID:  11320339
CoreID:  11320350
CoreID:  11320356
CoreID:  11320381
CoreID:  11320382
CoreID:  11320384
CoreID:  11320392
CoreID:  11320399
CoreID:  11320400
CoreID:  11320401
CoreID:  11320403
CoreID:  11320404
CoreID:  11320409
CoreID:  11320411
CoreID:  11320417
CoreID:  11320428
CoreID:  11320456
CoreID:  11320462
CoreID:  11320463
CoreID:  11320464
CoreID:  11320470
CoreID:  11320471
CoreID:  11320473
CoreID:  11320478
CoreID:  11320489
CoreID:  11320490
CoreID:  11320493
CoreID:  11320501
CoreID:  11320515
CoreID:  11320522
CoreID:  11320524
CoreID:  11320525
CoreID:  11320526
CoreID:  11320527
CoreID:  11320528
CoreID:  11320529
CoreID:  11320530
CoreID:  11320534
CoreID:  11320537
CoreID:  11320539
CoreID:  11320541
CoreID:  11320542
CoreID:  11320543
CoreID:  11320556
CoreID:  11320557
CoreID:  11320558
CoreID:  11320560
CoreID:  11320561
CoreID:  11320574
CoreID:  11320576
CoreID:  11320577
CoreID:  11320583
CoreID:  1

...

CoreID:  33756325
CoreID:  33756326
CoreID:  33756327
CoreID:  33756328
CoreID:  33756329
CoreID:  33756330
CoreID:  33756331
CoreID:  33756332
CoreID:  33756333
CoreID:  33756364
CoreID:  33756365
CoreID:  33756369
CoreID:  33756371
CoreID:  33756374
CoreID:  33756377
CoreID:  33756382
CoreID:  33756384
CoreID:  33756389
CoreID:  33756393
CoreID:  33756396
CoreID:  33756398
CoreID:  33756399
CoreID:  33756401
CoreID:  33756402
CoreID:  33756404
CoreID:  33756406
CoreID:  33756408
CoreID:  33756409
CoreID:  33756410
CoreID:  33756411
CoreID:  33756413
CoreID:  33756414
CoreID:  33756415
CoreID:  33756416
CoreID:  33756417
CoreID:  33756418
CoreID:  33756419
CoreID:  33756421
CoreID:  33756422
CoreID:  33756533
CoreID:  33756534
CoreID:  33756536
CoreID:  33756537
CoreID:  33756538
CoreID:  33756540
CoreID:  33756541
CoreID:  33756543
CoreID:  33756544
CoreID:  33756545
CoreID:  33756547
CoreID:  33756548
CoreID:  33756549
CoreID:  33756551
CoreID:  33756552
CoreID:  33756553
CoreID:  3

CoreID:  33757762
CoreID:  33757763
CoreID:  33757764
CoreID:  33757765
CoreID:  33757766
CoreID:  33757767
CoreID:  33757768
CoreID:  33757769
CoreID:  33757770
CoreID:  33757771
CoreID:  33757773
CoreID:  33757774
CoreID:  33757775
CoreID:  33757776
CoreID:  33757778
CoreID:  33757779
CoreID:  33757780
CoreID:  33757781
CoreID:  33757785
CoreID:  33757787
CoreID:  33757788
CoreID:  33757789
CoreID:  33757790
CoreID:  33757791
CoreID:  33757792
CoreID:  33757794
CoreID:  33757795
CoreID:  33757818
CoreID:  33757821
CoreID:  33757823
CoreID:  33757825
CoreID:  33757826
CoreID:  33757827
CoreID:  33757829
CoreID:  33757830
CoreID:  33757831
CoreID:  33757832
CoreID:  33757833
CoreID:  33757834
CoreID:  33757836
CoreID:  33757837
CoreID:  33757838
CoreID:  33757841
CoreID:  33757844
CoreID:  33757845
CoreID:  33757847
CoreID:  33757848
CoreID:  33757849
CoreID:  33757850
CoreID:  33757851
CoreID:  33757853
CoreID:  33757854
CoreID:  33757855
CoreID:  33757859
CoreID:  33757862
CoreID:  3

CoreID:  42032353
CoreID:  42032362
CoreID:  42032366
CoreID:  42032371
CoreID:  42032384
CoreID:  42032388
CoreID:  42032391
CoreID:  42032420
CoreID:  42032421
CoreID:  42032429
CoreID:  42032432
CoreID:  42032436
CoreID:  42032452
CoreID:  42032454
CoreID:  42032457
CoreID:  42032461
CoreID:  42032472
CoreID:  42032481
CoreID:  42032485
CoreID:  42032486
CoreID:  42032487
CoreID:  42032493
CoreID:  42032506
CoreID:  42032508
CoreID:  42032512
CoreID:  42032516
CoreID:  42032526
CoreID:  42032534
CoreID:  42032539
CoreID:  42032545
CoreID:  42032550
CoreID:  42032554
CoreID:  42032555
CoreID:  42032558
CoreID:  42032564
CoreID:  42032570
CoreID:  42032585
CoreID:  42032587
CoreID:  42032589
CoreID:  42032591
CoreID:  42032592
CoreID:  42032596
CoreID:  42032601
CoreID:  42032604
CoreID:  42032624
CoreID:  42032631
CoreID:  42032641
CoreID:  42032647
CoreID:  42032648
CoreID:  42032649
CoreID:  42032652
CoreID:  42032653
CoreID:  42032655
CoreID:  42032700
CoreID:  42032701
CoreID:  4

CoreID:  78068291
CoreID:  78068292
CoreID:  78068294
CoreID:  78068296
CoreID:  78068298
CoreID:  78068300
CoreID:  78068302
CoreID:  78068304
CoreID:  78068305
CoreID:  78068306
CoreID:  78068309
CoreID:  78068310
CoreID:  78068311
CoreID:  78068312
CoreID:  78068313
CoreID:  78068314
CoreID:  78068315
CoreID:  78068316
CoreID:  78068317
CoreID:  78068318
CoreID:  78068319
CoreID:  78068320
CoreID:  78068321
CoreID:  78068323
CoreID:  78068324
CoreID:  78068325
CoreID:  78068326
CoreID:  78068327
CoreID:  78068328
CoreID:  78068330
CoreID:  78068331
CoreID:  78068332
CoreID:  78068333
CoreID:  78068335
CoreID:  78068336
CoreID:  78068338
CoreID:  78068339
CoreID:  78068340
CoreID:  78068342
CoreID:  78068345
CoreID:  78068346
CoreID:  78068347
CoreID:  78068348
CoreID:  78068349
CoreID:  78068352
CoreID:  78068353
CoreID:  78068354
CoreID:  78068355
CoreID:  78068357
CoreID:  78068358
CoreID:  78068360
CoreID:  78068361
CoreID:  78068362
CoreID:  78068364
CoreID:  78068365
CoreID:  7

CoreID:  83144351
CoreID:  83144352
CoreID:  83144353
CoreID:  83144357
CoreID:  83144358
CoreID:  83144360
CoreID:  83144361
CoreID:  83144363
CoreID:  83144364
CoreID:  83144365
CoreID:  83144366
CoreID:  83144367
CoreID:  83144368
CoreID:  83144369
CoreID:  83144370
CoreID:  83144371
CoreID:  83144372
CoreID:  83144373
CoreID:  83144374
CoreID:  83144375
CoreID:  83144376
CoreID:  83144382
CoreID:  83144383
CoreID:  83144384
CoreID:  83144385
CoreID:  83144386
CoreID:  83144388
CoreID:  83144389
CoreID:  83144390
CoreID:  85123921
CoreID:  85123922
CoreID:  85123923
CoreID:  85123924
CoreID:  85123925
CoreID:  85123926
CoreID:  85123927
CoreID:  85123928
CoreID:  85123930
CoreID:  85123932
CoreID:  85123934
CoreID:  85123935
CoreID:  85123936
CoreID:  85123937
CoreID:  85123941
CoreID:  85123945
CoreID:  85123946
CoreID:  85123947
CoreID:  85123948
CoreID:  85123949
CoreID:  85123950
CoreID:  85123951
CoreID:  85123952
CoreID:  85123953
CoreID:  85123954
CoreID:  85123955
CoreID:  8

In [22]:
print(len(kein_autor))
print(len(kein_titel))
print(len(error_papers))

print(len(ergebnis_label))
print(len(ergebnis_tokens))
print(len(ergebnis_label_real))

106
13321
2
15550
15550
15550


# Evaluation

## Accuracy

In [25]:
acc = []
acc_100 = []
for i in range(len(ergebnis_tokens)):
    acc.append(accuracy_score(ergebnis_label[i], ergebnis_label_real[i]))
    if ergebnis_label[i]== ergebnis_label_real[i]:
        acc_100.append(1)
        
print("Accuracy: " + str(np.mean(acc)*100) + " %")
print("Completly correct classifications: " + str(len(acc_100)/len(acc)*100) + " %")

Accuracy: 96.55426828645383 %
Completly correct classifications: 0.13504823151125403 %


## Macro Jaccard Score

In [26]:
Jaccard_score = []
for p in range(len(ergebnis_label_real)):
    jac = jaccard_score(ergebnis_label_real[p], ergebnis_label[p] , average="macro")
    Jaccard_score.append(jac)
    
print("Macro Jaccard Score: " + str(np.mean(Jaccard_score)*100) + " %")


Macro Jaccard Score: 27.60339278754594 %


## Macro F1-Score

In [27]:
F1_score = []
for p in range(len(ergebnis_label_real)):
    f1 = f1_score(ergebnis_label_real[p], ergebnis_label[p] , average="macro")
    F1_score.append(f1)
    
print("Macro F1 Score: " + str(np.mean(F1_score)*100) + " %")


Macro F1 Score: 28.127084463729908 %
